In [ ]:
# Instalação da biblioteca para interagir com a API Gemini
!pip install google-generativeai

# Importação da biblioteca
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import google.generativeai as genai
import json
from google.colab import userdata

In [ ]:
API_KEY = userdata.get('GOOGLE_API_KEY')


# Autenticação (substitua pela sua chave de API)
genai.configure(api_key=API_KEY)

# Ex 07

In [ ]:


# Notícia (substitua pelo texto da notícia que você deseja resumir)
noticia = """
Celso Araujo Sampaio de Novais, de 41 anos, foi atingido por um tiro de fuzil nas costas e estava internado na Unidade de Terapia Intensiva (UTI) do Hospital Geral de Guarulhos.

A comunicação do óbito foi registrada no 7º DP da cidade, de acordo com a Secretaria de Segurança Pública de São Paulo.

Além de Sampaio, outras duas pessoas foram atingidas: um homem, de 39 anos, que também foi internado no Hospital Geral de Guarulhos, e uma mulher, de 28 anos, que recebeu atendimento médico e foi liberada em seguida – ela foi ouvida pelos policiais no local.

Foram ao menos 27 disparos efetuados por dois criminosos, segundo informações da perícia do Departamento de Homicídios e Proteção à Pessoa Humana (DHPP) da Polícia Civil, que investiga o caso.
"""

# Prompt com exemplos para guiar o resumo
prompt = f"""
**Instruções:** Resuma a notícia abaixo em 1 sentença usando suas próprias palavras.

**Exemplo 1:**
Notícia: "O governo anunciou hoje um novo plano de saúde para os cidadãos."
Resumo: "O governo lançou um novo plano de saúde."

**Exemplo 2:**
Notícia: "A empresa X anunciou que irá fechar 500 vagas de trabalho."
Resumo: "A empresa X irá demitir 500 funcionários."

**Notícia:**
{noticia}

**Resumo:**
"""

# Executando o prompt com o modelo Gemini
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)

# Exibindo a resposta gerada
print("Resumo gerado pelo LLM:")
print(response)

# Ex 08

In [ ]:
# Notícias escolhidas
noticia = """
    Notícia 1: O presidente da República, Luiz Inácio Lula da Silva, anunciou ontem que o governo vai investir R$ 1 bilhão em infraestrutura para o estado de São Paulo.
    Notícia 2: A empresa de tecnologia, Google, anunciou que vai criar 1.000 novos empregos no Brasil nos próximos dois anos.
    Notícia 3: O ministro da Educação, Abraham Bragança, disse que o governo vai aumentar o investimento em educação em 10% no próximo ano.
"""

# Prompt para o LLM
prompt = f"""Identifique as menções a diferentes entidades (pessoas, órgãos públicos, empresas, etc.) na seguinte notícia:"

**Notícia:**
{noticia}
"""

# Executando o prompt com o modelo Gemini
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)

# Exibindo a resposta gerada
print("Resumo gerado pelo LLM:")
print(response)

# Ex 09

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")

# Prompt para o LLM
prompt = f"""
Água mole em pedra dura, tanto bate até que fura.
"""*500

print(f"Quantidade de palavras: {len(prompt.split())}")

# Executando o prompt com o modelo Gemini
response = model.generate_content(prompt)

# Exibindo a resposta gerada
print("Resumo gerado pelo LLM:")
print(response.usage_metadata)

# Ex 11

In [ ]:
# Carregar o arquivo YAML
with open('roteiro.yml', 'r') as f:
    roteiro = yaml.safe_load(f)

model = genai.GenerativeModel("gemini-1.5-flash")

# Executar os prompts para cada cidade
for cidade, prompt in roteiro['roteiro'].items():
    print(f"**{cidade}**")
    resposta = model.generate_content(prompt['prompt'])
    print(resposta)
    print()

# Ex 12

In [ ]:
'''
roteiro:
  atenas:
    prompt: "Liste 3 pontos turísticos de Atenas, o número anual de visitantes e uma breve descrição de cada um. Faça com que a resposta seja um json. Os visitantes anuais são um número sem ponto ou vírgula. Evite o fórum romano. Exemplo: nome:acropole, visitantes_anuais:320000, desc:antigo sitio"
  roma:
    prompt: "Liste 3 pontos turísticos de Roma, o número anual de visitantes e uma breve descrição de cada um. Faça com que a resposta seja um json. Os visitantes anuais são um número sem ponto ou vírgula. Evite o fórum romano. Exemplo: nome:acropole, visitantes_anuais:320000, desc:antigo sitio"
 
'''


with open('roteiro.yml', 'r') as f:
    roteiro = yaml.safe_load(f)

# Definir a API do Gemini
model = genai.GenerativeModel("gemini-1.5-flash")

responseList = []
for cidade, prompt in roteiro['roteiro'].items():
    response = model.generate_content(prompt['prompt'])
    entrada = response.text
    entrada = entrada.replace("```json", "" , 1)
    entrada = entrada.replace("```", "" , 1)
    entrada = entrada.replace("\n","")
    entrada = entrada.replace(" ","", 4)


    responseList.append(entrada)
    print(entrada)


In [ ]:
dados = []


json_data_atenas = json.loads(responseList[0])
dados.append(json_data_atenas)

json_data_roma = json.loads(responseList[1])
dados.append(json_data_roma)

print(json_data_atenas)
print(json_data_roma)
print(dados)

In [ ]:

df = pd.DataFrame(dados[0])

df['cidade'] = 'Atenas'

df2 = pd.DataFrame(dados[1])
df2['cidade'] = 'Roma'

df_final = pd.concat ([df, df2])
print(df_final)


# Plotar o gráfico de barras horizontais
plt.figure(figsize=(12, 6))
cores = {'Roma': 'red', 'Atenas': 'blue'}  # Define cores para cada cidade
for cidade in df_final['cidade'].unique():
  plt.barh(df_final.loc[df_final['cidade'] == cidade, 'nome'], df_final.loc[df_final['cidade'] == cidade, 'visitantes_anuais'],
            label=cidade, color=cores.get(cidade, 'gray'))
plt.xlabel('Número de Visitantes (em milhões)')
plt.ylabel('Pontos Turísticos')
plt.title('Número de visitantes por ponto turístico')
plt.legend(loc='best')
plt.xticks([0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000, 9000000, 10000000], ['0', '1M', '2M', '3M', '4M', '5M', '6M', '7M', '8M', '9M', '10M'])
plt.show()